# Multi Agent-Systems
A Team is a group of agents that work together to achieve a common goal

In [3]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)
model_client_2 = "maybe Open Host"

# Single Agent Approach
A single agent is going to create a short story for us

In [4]:
from autogen_agentchat.agents import (AssistantAgent)

story_agent = AssistantAgent(
    name = 'story_agent',
    model_client=model_client,
    system_message='You are a creative writer. Generate a short story about a brave knight and a dragon.'
)

In [5]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 50 words',
        source='user'
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)

await test_simple_agent()

In a forgotten realm, Sir Cedric faced the dreaded dragon atop Mount Etherfire. As flames roared, Cedric saw the dragon's eyes held sorrow, not malice. With empathy, he approached, discovering a thorn causing its pain. He gently removed it, forging an unlikely friendship that saved the kingdom.


## Question: Who will review Output?
Basically, model created output, but now need another model to review generated output

# Multi Agent Team Approach
We will be using (3) agents to now write the story, each with its own work and mastery

In [6]:
plot_agent = AssistantAgent(
    name= 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on being a murder mystery detective chef."
)

character_agent = AssistantAgent(
    name='character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the chefs and the murderer in detail, including their motivations and background."
)

ending_agent = AssistantAgent(
    name='ending_writer',
    model_client=model_client,
    system_message="You write engaing endings. conclude the story with a twist."
)

## Teams: RoundRobinGroupChat
Team configuration where all agents share the same context and take terms responding in round-robin fashion

In [7]:
from autogen_agentchat.teams import RoundRobinGroupChat

# Create team 
# (only takes 3 turns, so will visit each agent once in this example)
team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent],
    max_turns=3
)
team.component_type



'team'

In [8]:
# Create Execution
async def test_team():
    task = TextMessage(
        content='Write a short story of a murder mystery chef. Keept it up to 50 words',
        source='user'
    )

    
    result = await team.run(task=task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')

# Run 
await test_team()

user : Write a short story of a murder mystery chef. Keept it up to 50 words
plot_writer : Chef Laurent discovered a peculiar spice in the stew—cyanide. As guests fell ill, he recalled rival sous-chef Pierre's recent dismissal. Laurent’s knife skills and keen taste identified the deadly twist, unveiling Pierre's plot. Police arrived, arresting Pierre, and Chef Laurent saved the evening, once again a culinary hero.
character_writer : Chef Laurent Ducasse: Renowned for his exquisite palate and relentless pursuit of culinary perfection, Chef Laurent is a maestro in the kitchen. Hailing from a lineage of celebrated French chefs, his motivation is to uphold his family's legacy, pushing the boundaries of gastronomy. His tall, lean figure and graying hair give him an air of authority. Despite his sometimes imperious demeanor, Laurent is deeply passionate about mentoring young chefs, seeing it as a way to inspire the next generation of culinary innovators.

Chef Remy "Renard" Leclair: Known fo

user : Write a short story of a murder mystery chef. Keept it up to 50 words
1. plot_writer : Chef Nico sensed something was off in the kitchen. The salt was replaced with cyanide. Sous-chef Carla lay motionless on the floor. Using his culinary skills, Nico traced the crime back to rival chef Marco, whose motive was to sabatoge Nico’s restaurant opening. Justice, like a dish, was served cold.


2. character_writer : Chef Nico Andrés was a culinary prodigy, born into a family of street vendors in Seville. His rise to fame was fueled by a fierce determination to transcend his humble beginnings. Known for his innovative use of spices, Nico aimed to open his own restaurant, a dream laced with both passion and pressure. His ambition was driven by a desire to make his mark in the world and prove himself beyond his familial ties.

Marco Delaney, a renowned chef hailing from a storied lineage of restaurateurs in Paris, felt a constant shadow of expectation looming over him. Though his culinary skills were unparalleled, Marco harbored deep-seated jealousy towards self-made chefs like Nico. His illustrious family background left little room for personal failure, and the thought of Nico's imminent success clawed at Marco’s insecurities.

Sous-chef Carla Rivera, a spirited and talented young chef from Valencia, admired Nico beyond measure. Her loyalty stemmed from a shared vision of culinary excellence and an unyielding belief in Nico’s potential. Carla was the embodiment of hope and dedication, an aspiring chef who saw the kitchen not just as a workplace, but as a canvas for dreams.

The murder was a result of Marco’s desperation. The pressure of his family's expectations weighed heavily on him, and seeing Nico as a threat pushed him over the edge. Marco orchestrated the crime meticulously, replacing the salt with cyanide, a plan as calculated and cold as a perfectly executed dish. In his mind, removing Nico would restore his standing in the culinary world and meet the unspoken demands of his lineage.

Detective Alina Guzmán, a no-nonsense investigator with a knack for detail, entered the scene with an open mind and an astute sense for deception. Her motivation was rooted in justice and a deep-seated disdain for unpunished crimes, especially those entwined with betrayal and envy. With a background in profiling, Alina methodically unraveled Marco’s plot, driven by a personal commitment to let no crime go unsolved.

As the investigation unfolded, the motivations of each character painted a picture of ambition, loyalty, and the destructive power of envy, culminating in a dramatic clash of personal dreams and dark intentions.


3. ending_writer : As Detective Alina closed the case, Chef Nico faced Marco in court, the truth finally aired. But as the verdict was announced, Nico's smirk revealed a hidden truth: he had known about Marco's plot all along and had secretly altered the recipe, saving Carla. His own restaurant then opened with unparalleled reviews, due entirely to Marco's scandal—a dark yet strategic twist no one anticipated.

## Run the Team under Observation
Running with stream and TaskResult allows dev to see individual messages to inspect model's thought process

In [9]:
from autogen_agentchat.base import TaskResult

# When running a script, use a async main function and call it from `asyncio.run(...)`
await team.reset()   # Reset the team for a new task

async for message in team.run_stream(task="Write a short poam about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='e72c2dd7-6509-4c75-bbca-341442e58ef0' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 1, 22, 13, 7, 14, 154113, tzinfo=datetime.timezone.utc) content='Write a short poam about the fall season.' type='TextMessage'
id='1dd1c8de-d16f-4526-b83c-56b3cd8115f0' source='plot_writer' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=150) metadata={} created_at=datetime.datetime(2026, 1, 22, 13, 7, 18, 833896, tzinfo=datetime.timezone.utc) content="In crisp embrace, the autumn sweeps,  \nWhere whispered secrets nature keeps.  \nLeaves pirouette in golden flight,  \nA gentle dance of waning light.  \n\nThe air is spiced with Earth's perfume,  \nA tapestry of hues in bloom.  \nScarlet, amber, rust entwined,  \nA rich mosaic, colors combined.  \n\nThe harvest moon casts silver beams,  \nOn orchards ripe with apple dreams.  \nAs fires crackle, stories spin,  \nOf summer's end and frost's begin.  \n\nIn this symphony of soft goodbyes,  \nThe fall breathe

## Team: SelectorGroupChat
NOTE: Checkout example and do on my own